New pumpistor - for our purposes

In [1]:
from matplotlib import pyplot as plt
from numpy import linspace
import numpy as np
import scipy
import scipy.optimize
import scipy.integrate
from pylab import *

pi = 3.14159
fi_0 = 2.07e-15/2/pi

I_c = 4e-6
C = 4e-12

In [2]:
def G_0(x,F,f): #x = w_p*t
    return(np.abs((np.cos(F + f*np.cos(x)))) /2/pi)

def G_1(x,F,f): #x = w_p*t
    return(np.abs((np.cos(F + f*np.cos(x)))) * np.cos(x)/pi )

def G_2(x,F,f): #x = w_p*t
    return( (np.cos(F + f*np.cos(x)) * np.cos(2*x))/pi )
        
def a_0(F,f):
    return(scipy.integrate.quad(lambda x: G_0(x,F,f) ,-pi,pi)[0])

def a_1(F,f):
    return(scipy.integrate.quad(lambda x: G_1(x,F,f) , -pi,pi)[0])

def a_2(F,f):
    return(scipy.integrate.quad(lambda x: G_2(x,F,f) , -pi,pi)[0])


def epsilon_0(F,f,a_0,a_1):
    return (a_0/np.cos(F))
#    return(1.- 0.25*f**2)
    
def epsilon_1(F,f,a_0,a_1):
    return( 0.5*a_1/np.cos(F) )
#    return(0.5*f*np.tan(F))
           
def L_J(F,f,a_0,a_1):
    return(fi_0/(I_c*np.cos(F)))

In [3]:
def V2divV1(F,f,a_0,a_1, w_p, w_s, Yr, Ym, Ysh):  #All admittances are expected at idler freq
    x = -1j*epsilon_1(F,f,a_0,a_1)*(1. + Yr/Ym).conjugate()/w_s/L_J(F,f,a_0,a_1)
    x = x/(-1j*epsilon_0(F,f,a_0,a_1)*(1. + Yr/Ym).conjugate()/(w_p - w_s)/L_J(F,f,a_0,a_1) - (Ysh + Yr + Ysh*Yr/Ym).conjugate())
    return(x)

def Y_SQ(F,f,a_0,a_1, w_p, w_s, Yr, Ym, Ysh):
    return( -1j* ( epsilon_0(F,f,a_0,a_1)/w_s - epsilon_1(F,f,a_0,a_1)*V2divV1(F,f,a_0,a_1, w_p, w_s, Yr, Ym, Ysh)/(w_p - w_s) ) / L_J(F,f,a_0,a_1) )

def Y_sh(w):
    return(1j*w*C)

def Y_JPA(F,f,a_0,a_1, w_p, w_s, Yr, Ym, Ysh):
    return(Y_JPA(F,f,a_0,a_1, w_p, w_s, Yr, Ym, Ysh) + Y_sh(w_s))

In [4]:

def Gain(F,f,a_0,a_1, w_p, w_s, Yr1, Ym1, Ysh1 ,Yr2 , Ym2, Ysh2): #1-signal, 2 - idler
    g = (-1./Yr1 + 1./Ym1 + 1./Y_JPA(F,f,a_0,a_1, w_p, w_s, Yr2, Ym2, Ysh2))/(1./Yr1 + 1./Ym1 + 1./Y_JPA(F,f,a_0,a_1, w_p, w_s, Yr2, Ym2, Ysh2))
    #g = (Yr1  - Y_SQ(F,f,a_0,a_1, w_p, w_s, Yr2, Ym2, Ysh2))/(Yr1  + Y_SQ(F,f,a_0,a_1, w_p, w_s, Yr2, Ym2, Ysh2))
    #alpha = (-1./Yr1 + 1./Ym1 + 1./Y_SQ(F,f,a_0,a_1, w_p, w_s, Yr2, Ym2, Ysh2))/(1./Yr1 + 1./Ym1 + 1./Y_SQ(F,f,a_0,a_1, w_p, w_s, Yr2, Ym2, Ysh2))
    return(20*np.log10(np.abs(g)))
    #return g

In [5]:
####Test
F = 0.2*pi
f = 0.16*pi
w_p = 14e9*6.28

a0 = a_0(F,f)
a1 = a_1(F,f)

Yreal = 1./50

w = linspace(3*6.28e9,12*6.28e9,1e5)


plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1e99*1j, Y_sh(w) ,Yreal , 1e99*1j, Y_sh(w_p - w) ))

plt.xlabel('w_s, Ghz')
plt.ylabel('G_s')
plt.title('')
plt.show()

RuntimeError: maximum recursion depth exceeded

In [5]:
#Optimization
#Yreal must be fond somewhere else
F = 0.2*pi
f = 0.16*pi
w_p = 14*6.28e9

a0 = a_0(F,f)
a1 = a_1(F,f)

Yreal = 1./50


length_w = 100
length_Y = 201
wStart = w_p/2
wStop = 6.28e9*7.5
YStart = -1/100.
YStop = -1/0.1


w = linspace(wStart, wStop,length_w)

def Y_quan(i):
    return(1j*(YStart + i*(YStop - YStart)/length_Y) )

def w_quan(n):
    return(wStart + n*(wStop - wStart)/length_w)

#
G_buf = []
G_idler = []
G = []
YmSi = []
YmId = []

Gbuf1 = 0 
Gbuf2 = 0
YsBuf = 0 + 0j
YiBuf = 0 + 0j

for k in range(length_w):
    #G_buf = []
    for l in range(length_Y):#Ym1
        for m in range(length_Y):#Ym2
                    Gbuf2 = Gain(F,f,a0,a1, w_p, w_quan(k), Yreal, Y_quan(l), Y_sh(w_quan(k)) ,Yreal , Y_quan(m), Y_sh(w_p - w_quan(k)) )
                    if (np.abs(Gbuf2-20.) < np.abs(Gbuf1-20.)):
                        if (k==0)&( Y_quan(l)!= Y_quan(m)):
                            continue
                        Gbuf1 = Gbuf2
                        YsBuf = Y_quan(l)
                        YiBuf = Y_quan(m)
                        #test
                        
                        
    G_buf.append(Gbuf1)
    G_idler.append(Gain(F,f,a0,a1, w_p, w_quan(k), Yreal, YiBuf, Y_sh(w_p - w_quan(k)) ,Yreal , YsBuf, Y_sh(w_quan(k))))
    YmSi.append(YsBuf.imag)
    YmId.append(YiBuf.imag)
    
    Gbuf1 = 0
    Gbuf2 = 0
    YsBuf = 0 + 0j
    YiBuf = 0 + 0j
    
plt.plot(w/6.28e9, G_buf)
plt.plot((w_p - w)/6.28e9, G_idler)
plt.show()


RuntimeError: maximum recursion depth exceeded

In [16]:
plt.plot(w/6.28e9, YmSi)
plt.plot((w_p - w)/6.28e9, YmId)
plt.show()

In [15]:
plt.plot(w/6.28e9, G_buf)
plt.plot((w_p - w)/6.28e9, G_idler)
plt.show()

In [15]:
F = 0.2*pi
f = 0.16*pi
w_p = 14*6.28e9

a0 = a_0(F,f)
a1 = a_1(F,f)

Yreal = 1./50


length_w = 100
length_Y = 101
wStart = w_p/2
wStop = 6.28e9*7.5
YStart = 0.05
YStop = -0.5


w = linspace(wStart, wStop,length_w)

def Y_quan(i):
    return(1j*(YStart + i*(YStop - YStart)/length_Y) )

def w_quan(n):
    return(wStart + n*(wStop - wStart)/length_w)

#
G_buf = []
G_idler = []
G = []
YmSi = []
YmId = []

Gbuf1 = 0 
Gbuf2 = 0
YsBuf = 0 + 0j
YiBuf = 0 + 0j
YsSmoother = 0j
YiSmoother = 0j
YsOld = 0j
YiOld = 0j

def IfSmooth(YsSmoother,YiSmoother, Ys,Yi,YsOld, YiOld):
    if ((np.abs(Ys - YsSmoother)<=np.abs(YsOld - YsSmoother))&(np.abs(Yi - YiSmoother)<=np.abs(YiOld - YiSmoother))):
        return True
    else:
        return False

for k in range(length_w):
    #G_buf = []
    for l in range(length_Y):#YmSi
        for m in range(length_Y):#YmId
                    if (k==0)&( Y_quan(l)!= Y_quan(m)):
                            continue
                            
                    Gbuf2 = Gain(F,f,a0,a1, w_p, w_quan(k), Yreal, Y_quan(l), Y_sh(w_quan(k)) ,Yreal , Y_quan(m), Y_sh(w_p - w_quan(k)) )
                    
                    if (k==0):
                        if ( np.abs(Gbuf2-18.) <  np.abs(Gbuf1-18.)):
                            Gbuf1 = Gbuf2
                            YsBuf = Y_quan(l)
                            YiBuf = Y_quan(m)
                            continue
                            
                    else:        
                        if (( np.abs(Gbuf2-18.) < 0.25)& IfSmooth(YsSmoother,YiSmoother,Y_quan(l), Y_quan(m),YsBuf,YiBuf)):
    
                            Gbuf1 = Gbuf2
                            YsBuf = Y_quan(l)
                            YiBuf = Y_quan(m)
                            #test
    YsSmoother = YsBuf
    YiSmoother = YiBuf                
                        
    G_buf.append(Gain(F,f,a0,a1, w_p, w_quan(k), Yreal, YsBuf, Y_sh(w_quan(k)) ,Yreal , YiBuf, Y_sh(w_p - w_quan(k)) ))
    G_idler.append(Gain(F,f,a0,a1, w_p, w_quan(k), Yreal, YiBuf, Y_sh(w_p - w_quan(k)) ,Yreal , YsBuf, Y_sh(w_quan(k))))
    YmSi.append(YsBuf.imag)
    YmId.append(YiBuf.imag)
    
    Gbuf2 = 0
    YsBuf = 1e99
    YiBuf = 1e99
    
plt.plot(w/6.28e9, G_buf)
plt.plot((w_p - w)/6.28e9, G_idler)
plt.show()

In [6]:
plt.plot(w/6.28e9, YmSi)

plt.plot((w_p - w)/6.28e9, YmId)
plt.show()

In [107]:
plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1e99*1j, Y_sh(w) ,Yreal , 1e99*1j, Y_sh(w_p - w) ))
plt.plot((w_p - w)/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1e99*1j, Y_sh(w) ,Yreal , 1e99*1j, Y_sh(w_p - w) ))
plt.plot(w/6.28e9, G_buf)
plt.plot((w_p - w)/6.28e9, G_idler)
plt.show()

In [109]:
np.savetxt('YmSi.txt',YmSi)
np.savetxt('YmId.txt',YmId)
np.savetxt('w.txt',w)
np.savetxt('w_p.txt',w_p - w)
np.savetxt('GainSignal.txt',G_buf)
np.savetxt('GainIdler.txt',G_idler)

In [5]:
###fitting 

def g(l1,l2, c1,c2,w): #2 - parallel
  #  z = 50.
   return( -1./(w*l1 - 1./w/c1 + 1./( 1./w/l2 - w*c2 )  )  )
   # return( -1./( 1./( 1./w/l1 - w*c1 ) + 1./( 1./w/l2 - w*c2 )  )  ) #####BEST
    #return( 1./( 1./(w*l1 - 1./w/c1 ) + 1./( w*l2 - 1./w/c2 )  )  )
 

def integrate(l1,l2, c1,c2):
    wLen = len(w)
    S = 0
    delta = (wStop-wStart)/wLen
    for x in range(wLen):
        S = S + delta*np.abs(YmSi[x] - g(l1,l2, c1,c2,w_quan(k)))
        S = S + delta*np.abs(YmId[x] - g(l1,l2, c1,c2,w_p - w_quan(k)))
    return S

ToCompare = 1e100
for c1 in linspace(1e-14, 1e-12, 1):
    for l1 in linspace(1e-10, 1e-9, 1):
        for c2 in linspace(1e-13, 1e-12, 1):
            for l2 in linspace(1e-10, 1e-9, 1):
                if(integrate(l1,l2, c1,c2) <ToCompare):
                    L1 = l1
                    C1 = c1
                    L2 = l2
                    C2 = c2
                    ToCompare = integrate(l1,l2, c1,c2)

print L1,L2,C1,C2
plt.plot(w/6.28e9, g(L1,L2,C1,C2,w))
#plt.plot(w/6.28e9, -6e-12*w + 0.2532)
plt.show()

NameError: global name 'w' is not defined

In [87]:
plt.plot(w/6.28e9, g(L1,L2,C1,C2,w))
#plt.plot(w/6.28e9, -6e-12*w + 0.2532)
plt.show()

In [9]:
def g(l1,l2, c1,c2,w): #2 - parallel
  #  z = 50.
   #return( -1./(w*l1 - 1./w/c1 + 1./( 1./w/l2 - w*c2 )  )  )
    return( -1./( 1./( 1./w/l1 - w*c1 ) + 1./( 1./w/l2 - w*c2 )  )  ) #####BEST
    #return( 1./( 1./(w*l1 - 1./w/c1 ) + 1./( w*l2 - 1./w/c2 )  )  )

In [11]:
F = 0.2*pi
f = 0.16*pi
w_p = 14*6.28e9

a0 = a_0(F,f)
a1 = a_1(F,f)

Yreal = 1./50

w = linspace(6.28e9 * 7, 6.28e9*7.5,1e5)
L1 = 4e-10
L2 = .92e-9
C1 = 9e-13
C2 = 1e-12
#plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1e99*1j, Y_sh(w) ,Yreal , 1e99*1j, Y_sh(w_p - w) ), 'b',linewidth = 3)
plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1j*g(L1,L2,C1,C2,w), Y_sh(w) ,Yreal , 1j*g(L1,L2,C1,C2,w_p - w), Y_sh(w_p - w) ),'r',linewidth = 3)

a = w_p - w
#plt.plot(a/6.28e9, Gain(F,f,a0,a1, w_p, a, Yreal, 1e99*1j, Y_sh(a) ,Yreal , 1e99*1j, Y_sh(w_p - a) ), 'b',linewidth = 3)
plt.plot(a/6.28e9, Gain(F,f,a0,a1, w_p, a, Yreal, 1j*g(L1,L2,C1,C2,a), Y_sh(a) ,Yreal , 1j*g(L1,L2,C1,C2,w_p - a), Y_sh(w_p - a) ),'r',linewidth = 3)

#plt.xlabel("Frequency, Ghz")
#plt.ylabel("Gain, dB")
plt.legend("ab")
plt.show()

In [31]:
plt.plot(w/6.28e9, g(L1,L2,C1,C2,w), 'b')
plt.plot(a/6.28e9, g(L1,L2,C1,C2,a), 'b')
plt.ylim(-0.5, 0.3)
#plt.plot(w/6.28e9, -6e-12*w + 0.2532)
plt.xlabel("Frequency, Ghz")
plt.ylabel("Admittance, 1j*Ohm^-1")
plt.show()

In [16]:
###6 elements fitting 

def g(l1,l2,l3, c1,c2,c3,w): #2 - parallel
  #  z = 50.
    return( -1./( 1./( 1./w/l1 - w*c1 ) + 1./( 1./w/l2 - w*c2 ) + + 1./( 1./w/l3 - w*c3 )  )  ) #####BEST
 

def integrate(l1,l2,l3, c1,c2,c3):
    wLen = len(w)
    S = 0
    delta = (wStop-wStart)/wLen
    for x in range(wLen):
        S = S + delta*np.abs(YmSi[x] - g(l1,l2,l3, c1,c2,c3,w_quan(k)))
        S = S + delta*np.abs(YmId[x] - g(l1,l2,l3, c1,c2,c3,w_p - w_quan(k)))
    return S

ToCompare = 1e100
length_cycle = 5

for c1 in linspace(1e-14, 1e-13, length_cycle):
    for l1 in linspace(5e-10, 5e-9, length_cycle):
        for c2 in linspace(1e-13, 1e-12, length_cycle):
            for l2 in linspace(1e-10, 1e-9, length_cycle):
                for c3 in linspace(1e-13, 5e-12, length_cycle):
                    for l3 in linspace(1e-10, 5e-9,length_cycle):
                        if(integrate(l1,l2,l3, c1,c2,c3) <ToCompare):
                            L1 = l1
                            C1 = c1
                            L2 = l2
                            C2 = c2
                            L3 = l3
                            C3 = c3
                            ToCompare = integrate(l1,l2,l3, c1,c2,c3)
    
print L1,L2,L3,C1,C2,C3
plt.plot(w/6.28e9, g(L1,L2,L3,C1,C2,C3,w))
#plt.plot(w/6.28e9, -6e-12*w + 0.2532)
plt.show()

5e-10 3.25e-10 1.325e-09 1e-13 3.25e-13 1.325e-12


In [19]:
#L1 = 5e-9
#L2 =1.4e-9
#L3 =1.1e-10 ###
#C1 = 5e-14
#C2 = .95e-12
#C3 = 4.6e-2 ###

L1 = .51e-9
L2 =1.4e-9 ###
L3 =.1e-09
C1 = 5e-14
C2 = .9e-12
C3 = 4.3e-12

plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1e99*1j, Y_sh(w) ,Yreal , 1e99*1j, Y_sh(w_p - w) ),'r')
plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1j*g(L1,L2,L3,C1,C2,C3,w), Y_sh(w) ,Yreal , 1j*g(L1,L2,L3,C1,C2,C3,w_p - w), Y_sh(w_p - w) ),'b')

a = w_p - w
plt.plot(a/6.28e9, Gain(F,f,a0,a1, w_p, a, Yreal, 1e99*1j, Y_sh(a) ,Yreal , 1e99*1j, Y_sh(w_p - a) ),'r')
plt.plot(a/6.28e9, Gain(F,f,a0,a1, w_p, a, Yreal, 1j*g(L1,L2,L3,C1,C2,C3,a), Y_sh(a) ,Yreal , 1j*g(L1,L2,L3,C1,C2,C3,w_p - a), Y_sh(w_p - a) ),'b')
plt.show()

C:\Users\Ivan\Anaconda\lib\site-packages\numpy\core\numeric.py:462: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [15]:
plt.plot(w/6.28e9, g(L1,L2,L3,C1,C2,C3,w))
plt.plot(a/6.28e9, g(L1,L2,L3,C1,C2,C3,a))
plt.show()


In [26]:
###just a test


t = Gain(F,f,a0,a1, w_p, w, Yreal, 1j*g(L1,L2,L3,C1,C2,C3,w), Y_sh(w) ,Yreal , 1j*g(L1,L2,L3,C1,C2,C3,w_p - w), Y_sh(w_p - w) )

delta = 1/(1 + 1j*50*g(L1,L2,L3,C1,C2,C3,w))

gain = delta + (1-delta)*t

t2 = Gain(F,f,a0,a1, w_p, a, Yreal, 1j*g(L1,L2,L3,C1,C2,C3,a), Y_sh(a) ,Yreal , 1j*g(L1,L2,L3,C1,C2,C3,w_p - a), Y_sh(w_p - a) )

delta2 = 1/(1 + 1j*50*g(L1,L2,L3,C1,C2,C3,a))

gain2 = delta + (1-delta)*t

plt.plot(w/6.28e9, 20*np.log10(np.abs(gain)))
plt.plot(a/6.28e9, 20*np.log10(np.abs(gain2)))
plt.show()


In [38]:
#4-tank scheme

def g(l1,l2,l3,l4, c1,c2,c3,c4,w): #2 - parallel
  #  z = 50.
    return( -1./( 1./( 1./w/l1 - w*c1 ) + 1./( 1./w/l2 - w*c2 ) +  1./( 1./w/l3 - w*c3 ) + 1./( 1./w/l4 - w*c4 )  )  ) #####BEST
 

def integrate(l1,l2,l3,l4, c1,c2,c3,c4):
    wLen = len(w)
    S = 0
    delta = (wStop-wStart)/wLen
    for x in range(wLen):
        S = S + delta*np.abs(YmSi[x] - g(l1,l2,l3,l4, c1,c2,c3,c4,w_quan(k)))
        S = S + delta*np.abs(YmId[x] - g(l1,l2,l3,l4, c1,c2,c3,c4,w_p - w_quan(k)))
    return S

ToCompare = 1e100
length_cycle = 4

for c1 in linspace(1e-15, 1e-14, length_cycle):
    for l1 in linspace(10e-10, 5e-9, length_cycle):
        for c2 in linspace(1e-12, 5e-12, length_cycle):
            for l2 in linspace(1e-9, 5e-9, length_cycle):
                for c3 in linspace(1e-12, 5e-12, length_cycle):
                    for l3 in linspace(1e-19, 10e-9,length_cycle):
                        for c4 in linspace(1e-12, 5e-12, length_cycle):
                            for l4 in linspace(1e-9, 10e-9,length_cycle):
                                if(integrate(l1,l2,l3,l4, c1,c2,c3,c4) <ToCompare):
                                    L1 = l1
                                    C1 = c1
                                    L2 = l2
                                    C2 = c2
                                    L3 = l3
                                    C3 = c3
                                    L4 = l4
                                    C4 = c4
                                    ToCompare = integrate(l1,l2,l3,l4, c1,c2,c3,c4)
    
print L1,L2,L3,L4,C1,C2,C3,C4
plt.plot(w/6.28e9, g(L1,L2,L3,L4,C1,C2,C3,C4,w))
#plt.plot(w/6.28e9, -6e-12*w + 0.2532)
plt.show()

2.33333333333e-09 2.33333333333e-09 3.3333333334e-09 7e-09 7e-15 1e-12 1e-12 1e-12


In [142]:
#L1 = .51e-9
#L2 =1.4e-9 ###
#L3 =.1e-09
#C1 = 5e-14
#C2 = .9e-12
#C3 = 4.3e-12

L1 = 1.85e-9 ###
L2 =2e-9 
L3 =1.5e-09  ####
L4 = .55e-9   ###
C1 = 5e-15   ###
C2 = 1.7e-12  
C3 = 2e-12  ###
C4 = 1.5e-12  ###


plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1e99*1j, Y_sh(w) ,Yreal , 1e99*1j, Y_sh(w_p - w) ),'r')
plt.plot(w/6.28e9, Gain(F,f,a0,a1, w_p, w, Yreal, 1j*g(L1,L2,L3,L4,C1,C2,C3,C4,w), Y_sh(w) ,Yreal , 1j*g(L1,L2,L3,L4,C1,C2,C3,C4,w_p - w), Y_sh(w_p - w) ),'b')

a = w_p - w
plt.plot(a/6.28e9, Gain(F,f,a0,a1, w_p, a, Yreal, 1e99*1j, Y_sh(a) ,Yreal , 1e99*1j, Y_sh(w_p - a) ),'r')
plt.plot(a/6.28e9, Gain(F,f,a0,a1, w_p, a, Yreal, 1j*g(L1,L2,L3,L4,C1,C2,C3,C4,a), Y_sh(a) ,Yreal , 1j*g(L1,L2,L3,L4,C1,C2,C3,C4,w_p - a), Y_sh(w_p - a) ),'b')
plt.show()

In [114]:
plt.plot(w/6.28e9, g(L1,L2,L3,L4,C1,C2,C3,C4,w))
plt.plot(a/6.28e9, g(L1,L2,L3,L4,C1,C2,C3,C4,a))
plt.show()